In [1]:
from nltk import PCFG, ProbabilisticProduction as pp, nonterminals, Nonterminal, Production, ProbabilisticMixIn
from nltk.parse.generate import generate
import itertools
import sys
import random

In [2]:
prods = ["P -> MONDAY [0.2] | TUESDAY [0.2] | WEDNESDAY [0.2] | THURSDAY [0.2] | FRIDAY [0.2]", 
         "MONDAY -> 'go-to-school' 'go-to-work' 'do-chores' 'do-homework' [0.125]",
         "MONDAY -> 'go-to-school' 'stay-for-tutoring' 'do-chores' 'do-homework' [0.125]",
         "MONDAY -> 'go-to-school' 'go-to-work' 'go-running' 'do-homework' [0.125]",
         "MONDAY -> 'go-to-school' 'stay-for-tutoring' 'go-running' 'do-homework' [0.125]",
         "MONDAY -> 'go-to-school' 'go-to-work' 'do-chores' 'play-videogames' [0.125]",
         "MONDAY -> 'go-to-school' 'stay-for-tutoring' 'do-chores' 'play-videogames' [0.125]",
         "MONDAY -> 'go-to-school' 'stay-for-tutoring' 'go-running' 'play-videogames' [0.250]",
         "TUESDAY -> 'go-to-work' 'go-to-school' 'go-to-store' 'do-homework' [0.125]",
         "TUESDAY -> 'go-to-work' 'go-to-school' 'do-homework' 'play-videogames' [0.125]",
         "TUESDAY -> 'go-to-school' 'stay-for-tutoring' 'go-to-store' 'do-homework' [0.125]",
         "TUESDAY -> 'go-to-school' 'stay-for-tutoring' 'do-homework' 'play-videogames' [0.125]",
         "TUESDAY -> 'go-to-work' 'go-to-school' 'go-to-store' 'play-videogames' [0.125]",
         "TUESDAY -> 'go-to-work' 'go-to-school' 'do-chores' 'play-videogames' [0.0625]",
         "TUESDAY -> 'go-to-school' 'stay-for-tutoring' 'go-to-store' 'play-videogames' [0.125]",
         "TUESDAY -> 'go-to-school' 'stay-for-tutoring' 'do-chores' 'play-videogames' [0.0625]",
         "TUESDAY -> 'go-to-work' 'go-to-school' 'go-running' 'play-videogames' [0.0625]",
         "TUESDAY -> 'go-to-school' 'stay-for-tutoring' 'go-running' 'play-videogames' [0.0625]",
         "WEDNESDAY -> 'go-to-school' 'do-homework' 'go-to-work' 'do-chores' [0.250]",
         "WEDNESDAY -> 'go-to-school' 'do-homework' 'do-chores' 'play-videogames' [0.125]",
         "WEDNESDAY -> 'go-to-school' 'do-homework' 'go-running' 'do-chores' [0.125]",
         "WEDNESDAY -> 'go-to-school' 'stay-for-tutoring' 'go-to-work' 'do-chores' [0.125]",
         "WEDNESDAY -> 'go-to-school' 'stay-for-tutoring' 'play-videogames' 'do-chores' [0.125]",
         "WEDNESDAY -> 'go-to-school' 'go-to-work' 'go-running' 'do-chores' [0.125]",
         "WEDNESDAY -> 'go-to-school' 'stay-for-tutoring' 'go-running' 'do-chores' [0.125]",
        ]

In [3]:
test = PCFG.fromstring(prods)

[P -> WEDNESDAY [0.2]]


In [4]:
def sample_from_grammar(grammar, start=None, depth=None, n=None):
    """
    This is based off of the generate function in nltk for CFG. This uses ancestral
    sampling to generate one possible plan/sentence rather than all possible 
    plans/sentences. 
    see http://www.nltk.org/_modules/nltk/parse/generate.html#generate

    :param grammar: The Grammar used to generate sentences.
    :param start: The Nonterminal from which to start generate sentences.
    :param depth: The maximal depth of the generated tree.
    :param n: The maximum number of sentences to return.
    :return: An iterator of lists of terminal tokens.
    """
    if not start:
        start = grammar.start()
    if depth is None:
        depth = sys.maxsize

    iter = _sample_all(grammar, [start], depth)

    if n:
        iter = itertools.islice(iter, n)

    return iter

def _sample_all(grammar, items, depth):
    if items:
        try:
            for frag1 in _sample_one(grammar, items[0], depth):
                for frag2 in _sample_all(grammar, items[1:], depth):
                    yield frag1 + frag2
        except RuntimeError as _error:
            if _error.message == "maximum recursion depth exceeded":
                # Helpful error message while still showing the recursion stack.
                raise RuntimeError(
                    "The grammar has rule(s) that yield infinite recursion!!"
                )
            else:
                raise
    else:
        yield []


def _sample_one(grammar, item, depth):
    if depth > 0:
        if isinstance(item, Nonterminal):
            prod = random.choices([x for x in grammar.productions(lhs=item)],[x.prob() for x in test.productions(lhs=item)])[0]
            for frag in _sample_all(grammar, prod.rhs(), depth - 1):
                yield frag
        else:
            yield [item]


In [15]:
for s in sample_from_grammar(test,Nonterminal("MONDAY")):
    print(s)

['go-to-school', 'go-to-work', 'do-chores', 'do-homework']


In [6]:
M = Nonterminal("MONDAY")
l = ["go-to-school","go-to-work","do-chores","do-homework"]
prod = pp(M,l, prob=0.5)

print(prod)

MONDAY -> 'go-to-school' 'go-to-work' 'do-chores' 'do-homework' [0.5]


In [7]:
current_state = {'raining': 1, 'work-today': 1, 'need-groceries': 0,
                'have-homework': 1, 'found-movie': 0, 'went-to-school': 0,
                'went-to-work': 0, 'did-chores': 0, 'did-homework': 0,
                'stayed-for-tutoring': 0, 'ran': 0, 'played-videogames': 0,
                'went-to-store': 0, 'watched-movie': 0}


In [ ]:
class PSDG(PCFG):
    def __init__(self, start, productions, calculate_leftcorners=True):
        PCFG.__init__(self, start, productions, calculate_leftcorners)
    
    def update_prods(state):
        for i in self._productions:
            i.update_prob(state)

In [8]:
 class PSDProduction(Production,ProbabilisticMixIn):
    """
    Edited version of nltk's ProbablisticProduction class
    See http://www.nltk.org/_modules/nltk/grammar.html#ProbabilisticProduction
    
    This was edited to contain a precondition function and to be able to update
    the probability of production given a new state. The probability passed into
    the constructor represents the probability of the production if the passed
    state satisfies the preconditions.
    

    :see also: ``Production``
    """

    def __init__(self, lhs, rhs,current_s,precond, t_prob = 1.0):
        self.precond = precond
        self.t_prob = t_prob
        if self.precond(current_s):
            p = t_prob
        else:
            p = 1 - t_prob
            
        ProbabilisticMixIn.__init__(self, prob = p)
        Production.__init__(self, lhs, rhs)

    def __str__(self):
        return super().__str__() + (
            " [1.0]" if (self.prob() == 1.0) else " [%g]" % self.prob()
        )

    def __eq__(self, other):
        return (
            type(self) == type(other)
            and self._lhs == other._lhs
            and self._rhs == other._rhs
            and self.prob() == other.prob()
        )

    def __ne__(self, other):
        return not self == other

    def __hash__(self):
        return hash((self._lhs, self._rhs, self.prob()))
    
    def update_prob(state):
        if self.precond(state):
            p = t_prob
        else:
            p = 1 - t_prob
        self.set_prob(p)

In [9]:
def work_homework_raining(state):
    if state['raining']:
        if state['have-homework']:
            if state['work-today']:
                return True
    return False

In [10]:
M = Nonterminal("MONDAY")
l = ["go-to-school","go-to-work","do-chores","do-homework"]
prod = PSDProduction(M,l,current_state,work_homework_raining)

print(prod)

MONDAY -> 'go-to-school' 'go-to-work' 'do-chores' 'do-homework' [1.0]
